Germ Game
---
Two players against each other

In [1]:
import numpy as np
import pickle
import sys
from copy import deepcopy

In [2]:
BOARD_ROWS = 7
BOARD_COLS = 7

### Board State
---
Reflect & Judge the state

2 players p1 and p2; p1 uses symbol 1 and p2 uses symbol 2, vacancy as 0

In [3]:
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = 2
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1
    
    # get unique hash of current board state
    def getHash(self, position):
        next_board = deepcopy(self.board)
        p = position.copy()
        
        if p[2]-p[0]<0:
            for i in range(3):
                for j in range(3):
                    next_board[i][j],next_board[6-i][j] = next_board[6-i][j],next_board[i][j]
            p[0]=6-p[0]
            p[2]=6-p[2]

        if p[3]-p[1]<0:
            for i in range(7):
                for j in range(7):
                    next_board[i][j],next_board[i][6-j] = next_board[i][6-j],next_board[i][j]
            p[1]=6-p[1]
            p[3]=6-p[3]

        if p[2]-p[0]<p[3]-p[1]:
            for i in range(7):
                for j in range(i+1,7):
                    next_board[i][j],next_board[j][i] = next_board[j][i],next_board[i][j]
            p[0],p[1] = p[1],p[0]
            p[2],p[3] = p[3],p[2]
            
        s1 = s2 = s3 = 0
        e2 = e3 = 0
        shape = 0

        for dx in range(-3,4):
            for dy in range(-3,4):
                if not dx and not dy:
                    continue
                if p[0]+dx<0 or p[0]+dx>=BOARD_ROWS or p[1]+dy<0 or p[1]+dy>=BOARD_COLS:
                    continue

                if next_board[p[0]+dx][p[1]+dy]!=3-self.playerSymbol:
                    continue
                if max(abs(dx),abs(dy))==1:
                    s1=1
                if max(abs(dx),abs(dy))==2:
                    s2=1
                if max(abs(dx),abs(dy))==3:
                    s3=1

        for dx in range(-3,4):
            for dy in range(-3,4):
                if abs(dx)<=1 and abs(dy)<=1:
                    continue
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    continue

                if next_board[p[2]+dx][p[3]+dy]!=3-self.playerSymbol:
                    continue
                if max(abs(dx),abs(dy))==2:
                    e2=1
                if max(abs(dx),abs(dy))==3:
                    e3=1

        for dx in range(-1,2):
            for dy in range(-1,2):
                if not dx and not dy:
                    continue

                shape = shape<<1
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    shape = shape|1
                elif next_board[p[2]+dx][p[3]+dy]:
                    shape = shape|1

        # print(typ,s1,s2,s3,e2,e3,shape)

        if(p[3]-p[1]<2):
            return ((p[3]-p[1])<<12)+((s1|s2)<<11)+(s3<<10)+(e2<<9)+(e3<<8)+shape+1
        return ((p[3]-p[1])<<12)+(s1<<11)+(s2<<10)+(e2<<9)+(e3<<8)+shape+1
    
    def cantmove(self):
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    self.board[i, j] = 3 - self.playerSymbol
        return None
    
    def winner(self):
        # end
        if sum(map(sum,list(map(lambda row: list(map(lambda x: abs(2*x-3) if x!=0 else 0, row)), self.board)))) == BOARD_ROWS*BOARD_COLS:
            self.isEnd = True
            if sum(map(sum,list(map(lambda row: list(map(lambda x: 2*x-3 if x!=0 else 0, row)), self.board)))) < 0:
                return 1
            else:
                return 2
        # not end
        self.isEnd = False
        return None
    
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == self.playerSymbol:
                    for ii in range(-2,3):
                        for jj in range(-2,3):
                            if ii == 0 and jj == 0:
                                continue
                            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                                continue
                            if self.board[i + ii, j + jj] == 0:
                                positions.append([i, j, i + ii, j + jj])  # need to be tuple
        return positions
    
    def updateState(self, position):
        ii = position[2] - position[0]
        jj = position[3] - position[1]
        if max(abs(ii), abs(jj)) == 2:
            self.board[position[0], position[1]] = 0
        
        dx1 = [-1, -1, -1, 0, 0, 1, 1, 1]
        dy1 = [-1, 0, 1, -1, 1, -1, 0, 1]
        i, j = position[2:4]
        self.board[i, j] = self.playerSymbol
        for ii, jj in zip(dx1, dy1):
            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                continue
            if self.board[i + ii, j + jj] == 3 - self.playerSymbol:
                self.board[i + ii, j + jj] = self.playerSymbol
            
        # switch to another player
        self.playerSymbol = 3 - self.playerSymbol
    
    # only when game ends
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        """self.p1.feedReward(-result)
        self.p2.feedReward(result)"""
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        else:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
    
    # board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = 2
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 2
    
    def play(self, rounds=100):
        for i in range(rounds):
            if i%10 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:
                # Player 1
                positions = self.availablePositions()
                if not positions:
                    self.cantmove()
                else:
                    p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                    # take action and upate board state
                    self.updateState(p1_action)
                    # print("yay")
                    # print(positions)
                    # print(self.playerSymbol)
                    # print(p1_action)
                # self.showBoard()
                board_hash = self.getHash(p1_action)
                self.p1.addState(board_hash)
                # check board status if it is end
                win = self.winner()
                if win is not None:
                    # self.showBoard()
                    # ended with p1 either win or draw
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    positions = self.availablePositions()
                    if not positions:
                        self.cantmove()
                    else:
                        p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                        self.updateState(p2_action)
                        # print("aya")
                        # print(positions)
                        # print(self.playerSymbol)
                        # print(p2_action)
                    # self.showBoard()
                    board_hash = self.getHash(p2_action)
                    self.p2.addState(board_hash)
                    
                    win = self.winner()
                    if win is not None:
                        # self.showBoard()
                        # ended with p2 either win or draw
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break
    
    # play with human
    def play2(self):
        while not self.isEnd:
            # Player 1
            positions = self.availablePositions()
            if not positions:
                self.cantmove()
            else:
                # human
                # p1_action = self.p1.chooseAction(positions)
                # computer
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                # take action and upate board state
                self.updateState(p1_action)
            self.showBoard()
            # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                else:
                    print(self.p2.name, "wins!")
                self.reset()
                break

            else:
                # Player 2
                positions = self.availablePositions()
                if not positions:
                    self.cantmove()
                else:
                    # human
                    # p2_action = self.p2.chooseAction(positions)
                    # computer
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    # take action and upate board state
                    self.updateState(p2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == 1:
                        print(self.p1.name, "wins!")
                    else:
                        print(self.p2.name, "wins!")
                    print()
                    self.reset()
                    break

    def showBoard(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == 2:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('------------------------------')    

In [4]:
class Player:
    def __init__(self, name, exp_rate=0.1):
        self.name = name
        self.states = []  # record all positions taken
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.95
        self.states_value = {}  # state -> value
    
    def getHash(self, current_board, position, symbol):
        next_board = deepcopy(current_board)
        p = position.copy()
        
        if p[2]-p[0]<0:
            for i in range(3):
                for j in range(3):
                    next_board[i][j],next_board[6-i][j] = next_board[6-i][j],next_board[i][j]
            p[0]=6-p[0]
            p[2]=6-p[2]

        if p[3]-p[1]<0:
            for i in range(7):
                for j in range(7):
                    next_board[i][j],next_board[i][6-j] = next_board[i][6-j],next_board[i][j]
            p[1]=6-p[1]
            p[3]=6-p[3]

        if p[2]-p[0]<p[3]-p[1]:
            for i in range(7):
                for j in range(i+1,7):
                    next_board[i][j],next_board[j][i] = next_board[j][i],next_board[i][j]
            p[0],p[1] = p[1],p[0]
            p[2],p[3] = p[3],p[2]
            
        s1 = s2 = s3 = 0
        e2 = e3 = 0
        shape = 0

        for dx in range(-3,4):
            for dy in range(-3,4):
                if not dx and not dy:
                    continue
                if p[0]+dx<0 or p[0]+dx>=BOARD_ROWS or p[1]+dy<0 or p[1]+dy>=BOARD_COLS:
                    continue

                if next_board[p[0]+dx][p[1]+dy]!=3-symbol:
                    continue
                if max(abs(dx),abs(dy))==1:
                    s1=1
                if max(abs(dx),abs(dy))==2:
                    s2=1
                if max(abs(dx),abs(dy))==3:
                    s3=1

        for dx in range(-3,4):
            for dy in range(-3,4):
                if abs(dx)<=1 and abs(dy)<=1:
                    continue
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    continue

                if next_board[p[2]+dx][p[3]+dy]!=3-symbol:
                    continue
                if max(abs(dx),abs(dy))==2:
                    e2=1
                if max(abs(dx),abs(dy))==3:
                    e3=1

        for dx in range(-1,2):
            for dy in range(-1,2):
                if not dx and not dy:
                    continue

                shape = shape<<1
                if p[2]+dx<0 or p[2]+dx>=BOARD_ROWS or p[3]+dy<0 or p[3]+dy>=BOARD_COLS:
                    shape = shape|1
                elif next_board[p[2]+dx][p[3]+dy]:
                    shape = shape|1

        # print(typ,s1,s2,s3,e2,e3,shape)

        if(p[3]-p[1]<2):
            return ((p[3]-p[1])<<12)+((s1|s2)<<11)+(s3<<10)+(e2<<9)+(e3<<8)+shape+1
        return ((p[3]-p[1])<<12)+(s1<<11)+(s2<<10)+(e2<<9)+(e3<<8)+shape+1
    
    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                # print("p", p)
                boardHash = self.getHash(current_board, p, symbol)
                # print("boardHash", boardHash, self.states_value.get(boardHash))
                value = 0 if self.states_value.get(boardHash) is None else self.states_value.get(boardHash)
                # print("value", value, value_max)
                if value >= value_max:
                    value_max = value
                    action = p
        # print("{} takes action {}".format(self.name, action))
                    # print("action", action)
        return action
    
    # append a hash state
    def addState(self, state):
        self.states.append(state)
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr*(self.decay_gamma*reward - self.states_value[st])
            reward = self.states_value[st]
            
    def reset(self):
        self.states = []
        
    def savePolicy(self, rounds):
        fw = open('policy2_legr_' + str(self.lr) + '_' + str(self.exp_rate) + '_' + str(self.decay_gamma) + '_' + str(rounds) + '_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file,'rb')
        self.states_value = pickle.load(fr)
        fr.close()

In [5]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name 
    
    def chooseAction(self, positions):
        while True:
            # for x in positions:
                # print(x)
            row1 = int(input("Input your action row1:"))
            col1 = int(input("Input your action col1:"))
            row2 = int(input("Input your action row2:"))
            col2 = int(input("Input your action col2:"))
            action = (row1, col1, row2, col2)
            if action in positions:
                return action
            else:
                sys.exit(1)
    
    # append a hash state
    def addState(self, state):
        pass
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
            
    def reset(self):
        pass

### Training

In [16]:
p1 = Player("p1")
p2 = Player("p2")

st = State(p1, p2)

In [18]:
print("training...")
st.play(50000)

p1.savePolicy(100000)
p2.savePolicy(100000)

training...
Rounds 0
Rounds 10
Rounds 20
Rounds 30
Rounds 40
Rounds 50
Rounds 60
Rounds 70
Rounds 80
Rounds 90
Rounds 100
Rounds 110
Rounds 120
Rounds 130
Rounds 140
Rounds 150
Rounds 160
Rounds 170
Rounds 180
Rounds 190
Rounds 200
Rounds 210
Rounds 220
Rounds 230
Rounds 240
Rounds 250
Rounds 260
Rounds 270
Rounds 280
Rounds 290
Rounds 300
Rounds 310
Rounds 320
Rounds 330
Rounds 340
Rounds 350
Rounds 360
Rounds 370
Rounds 380
Rounds 390
Rounds 400
Rounds 410
Rounds 420
Rounds 430
Rounds 440
Rounds 450
Rounds 460
Rounds 470
Rounds 480
Rounds 490
Rounds 500
Rounds 510
Rounds 520
Rounds 530
Rounds 540
Rounds 550
Rounds 560
Rounds 570
Rounds 580
Rounds 590
Rounds 600
Rounds 610
Rounds 620
Rounds 630
Rounds 640
Rounds 650
Rounds 660
Rounds 670
Rounds 680
Rounds 690
Rounds 700
Rounds 710
Rounds 720
Rounds 730
Rounds 740
Rounds 750
Rounds 760
Rounds 770
Rounds 780
Rounds 790
Rounds 800
Rounds 810
Rounds 820
Rounds 830
Rounds 840
Rounds 850
Rounds 860
Rounds 870
Rounds 880
Rounds 890
Rounds 90

Rounds 6910
Rounds 6920
Rounds 6930
Rounds 6940
Rounds 6950
Rounds 6960
Rounds 6970
Rounds 6980
Rounds 6990
Rounds 7000
Rounds 7010
Rounds 7020
Rounds 7030
Rounds 7040
Rounds 7050
Rounds 7060
Rounds 7070
Rounds 7080
Rounds 7090
Rounds 7100
Rounds 7110
Rounds 7120
Rounds 7130
Rounds 7140
Rounds 7150
Rounds 7160
Rounds 7170
Rounds 7180
Rounds 7190
Rounds 7200
Rounds 7210
Rounds 7220
Rounds 7230
Rounds 7240
Rounds 7250
Rounds 7260
Rounds 7270
Rounds 7280
Rounds 7290
Rounds 7300
Rounds 7310
Rounds 7320
Rounds 7330
Rounds 7340
Rounds 7350
Rounds 7360
Rounds 7370
Rounds 7380
Rounds 7390
Rounds 7400
Rounds 7410
Rounds 7420
Rounds 7430
Rounds 7440
Rounds 7450
Rounds 7460
Rounds 7470
Rounds 7480
Rounds 7490
Rounds 7500
Rounds 7510
Rounds 7520
Rounds 7530
Rounds 7540
Rounds 7550
Rounds 7560
Rounds 7570
Rounds 7580
Rounds 7590
Rounds 7600
Rounds 7610
Rounds 7620
Rounds 7630
Rounds 7640
Rounds 7650
Rounds 7660
Rounds 7670
Rounds 7680
Rounds 7690
Rounds 7700
Rounds 7710
Rounds 7720
Rounds 7730
Roun

Rounds 13450
Rounds 13460
Rounds 13470
Rounds 13480
Rounds 13490
Rounds 13500
Rounds 13510
Rounds 13520
Rounds 13530
Rounds 13540
Rounds 13550
Rounds 13560
Rounds 13570
Rounds 13580
Rounds 13590
Rounds 13600
Rounds 13610
Rounds 13620
Rounds 13630
Rounds 13640
Rounds 13650
Rounds 13660
Rounds 13670
Rounds 13680
Rounds 13690
Rounds 13700
Rounds 13710
Rounds 13720
Rounds 13730
Rounds 13740
Rounds 13750
Rounds 13760
Rounds 13770
Rounds 13780
Rounds 13790
Rounds 13800
Rounds 13810
Rounds 13820
Rounds 13830
Rounds 13840
Rounds 13850
Rounds 13860
Rounds 13870
Rounds 13880
Rounds 13890
Rounds 13900
Rounds 13910
Rounds 13920
Rounds 13930
Rounds 13940
Rounds 13950
Rounds 13960
Rounds 13970
Rounds 13980
Rounds 13990
Rounds 14000
Rounds 14010
Rounds 14020
Rounds 14030
Rounds 14040
Rounds 14050
Rounds 14060
Rounds 14070
Rounds 14080
Rounds 14090
Rounds 14100
Rounds 14110
Rounds 14120
Rounds 14130
Rounds 14140
Rounds 14150
Rounds 14160
Rounds 14170
Rounds 14180
Rounds 14190
Rounds 14200
Rounds 14210

Rounds 19780
Rounds 19790
Rounds 19800
Rounds 19810
Rounds 19820
Rounds 19830
Rounds 19840
Rounds 19850
Rounds 19860
Rounds 19870
Rounds 19880
Rounds 19890
Rounds 19900
Rounds 19910
Rounds 19920
Rounds 19930
Rounds 19940
Rounds 19950
Rounds 19960
Rounds 19970
Rounds 19980
Rounds 19990
Rounds 20000
Rounds 20010
Rounds 20020
Rounds 20030
Rounds 20040
Rounds 20050
Rounds 20060
Rounds 20070
Rounds 20080
Rounds 20090
Rounds 20100
Rounds 20110
Rounds 20120
Rounds 20130
Rounds 20140
Rounds 20150
Rounds 20160
Rounds 20170
Rounds 20180
Rounds 20190
Rounds 20200
Rounds 20210
Rounds 20220
Rounds 20230
Rounds 20240
Rounds 20250
Rounds 20260
Rounds 20270
Rounds 20280
Rounds 20290
Rounds 20300
Rounds 20310
Rounds 20320
Rounds 20330
Rounds 20340
Rounds 20350
Rounds 20360
Rounds 20370
Rounds 20380
Rounds 20390
Rounds 20400
Rounds 20410
Rounds 20420
Rounds 20430
Rounds 20440
Rounds 20450
Rounds 20460
Rounds 20470
Rounds 20480
Rounds 20490
Rounds 20500
Rounds 20510
Rounds 20520
Rounds 20530
Rounds 20540

Rounds 26100
Rounds 26110
Rounds 26120
Rounds 26130
Rounds 26140
Rounds 26150
Rounds 26160
Rounds 26170
Rounds 26180
Rounds 26190
Rounds 26200
Rounds 26210
Rounds 26220
Rounds 26230
Rounds 26240
Rounds 26250
Rounds 26260
Rounds 26270
Rounds 26280
Rounds 26290
Rounds 26300
Rounds 26310
Rounds 26320
Rounds 26330
Rounds 26340
Rounds 26350
Rounds 26360
Rounds 26370
Rounds 26380
Rounds 26390
Rounds 26400
Rounds 26410
Rounds 26420
Rounds 26430
Rounds 26440
Rounds 26450
Rounds 26460
Rounds 26470
Rounds 26480
Rounds 26490
Rounds 26500
Rounds 26510
Rounds 26520
Rounds 26530
Rounds 26540
Rounds 26550
Rounds 26560
Rounds 26570
Rounds 26580
Rounds 26590
Rounds 26600
Rounds 26610
Rounds 26620
Rounds 26630
Rounds 26640
Rounds 26650
Rounds 26660
Rounds 26670
Rounds 26680
Rounds 26690
Rounds 26700
Rounds 26710
Rounds 26720
Rounds 26730
Rounds 26740
Rounds 26750
Rounds 26760
Rounds 26770
Rounds 26780
Rounds 26790
Rounds 26800
Rounds 26810
Rounds 26820
Rounds 26830
Rounds 26840
Rounds 26850
Rounds 26860

Rounds 32400
Rounds 32410
Rounds 32420
Rounds 32430
Rounds 32440
Rounds 32450
Rounds 32460
Rounds 32470
Rounds 32480
Rounds 32490
Rounds 32500
Rounds 32510
Rounds 32520
Rounds 32530
Rounds 32540
Rounds 32550
Rounds 32560
Rounds 32570
Rounds 32580
Rounds 32590
Rounds 32600
Rounds 32610
Rounds 32620
Rounds 32630
Rounds 32640
Rounds 32650
Rounds 32660
Rounds 32670
Rounds 32680
Rounds 32690
Rounds 32700
Rounds 32710
Rounds 32720
Rounds 32730
Rounds 32740
Rounds 32750
Rounds 32760
Rounds 32770
Rounds 32780
Rounds 32790
Rounds 32800
Rounds 32810
Rounds 32820
Rounds 32830
Rounds 32840
Rounds 32850
Rounds 32860
Rounds 32870
Rounds 32880
Rounds 32890
Rounds 32900
Rounds 32910
Rounds 32920
Rounds 32930
Rounds 32940
Rounds 32950
Rounds 32960
Rounds 32970
Rounds 32980
Rounds 32990
Rounds 33000
Rounds 33010
Rounds 33020
Rounds 33030
Rounds 33040
Rounds 33050
Rounds 33060
Rounds 33070
Rounds 33080
Rounds 33090
Rounds 33100
Rounds 33110
Rounds 33120
Rounds 33130
Rounds 33140
Rounds 33150
Rounds 33160

Rounds 38740
Rounds 38750
Rounds 38760
Rounds 38770
Rounds 38780
Rounds 38790
Rounds 38800
Rounds 38810
Rounds 38820
Rounds 38830
Rounds 38840
Rounds 38850
Rounds 38860
Rounds 38870
Rounds 38880
Rounds 38890
Rounds 38900
Rounds 38910
Rounds 38920
Rounds 38930
Rounds 38940
Rounds 38950
Rounds 38960
Rounds 38970
Rounds 38980
Rounds 38990
Rounds 39000
Rounds 39010
Rounds 39020
Rounds 39030
Rounds 39040
Rounds 39050
Rounds 39060
Rounds 39070
Rounds 39080
Rounds 39090
Rounds 39100
Rounds 39110
Rounds 39120
Rounds 39130
Rounds 39140
Rounds 39150
Rounds 39160
Rounds 39170
Rounds 39180
Rounds 39190
Rounds 39200
Rounds 39210
Rounds 39220
Rounds 39230
Rounds 39240
Rounds 39250
Rounds 39260
Rounds 39270
Rounds 39280
Rounds 39290
Rounds 39300
Rounds 39310
Rounds 39320
Rounds 39330
Rounds 39340
Rounds 39350
Rounds 39360
Rounds 39370
Rounds 39380
Rounds 39390
Rounds 39400
Rounds 39410
Rounds 39420
Rounds 39430
Rounds 39440
Rounds 39450
Rounds 39460
Rounds 39470
Rounds 39480
Rounds 39490
Rounds 39500

Rounds 45060
Rounds 45070
Rounds 45080
Rounds 45090
Rounds 45100
Rounds 45110
Rounds 45120
Rounds 45130
Rounds 45140
Rounds 45150
Rounds 45160
Rounds 45170
Rounds 45180
Rounds 45190
Rounds 45200
Rounds 45210
Rounds 45220
Rounds 45230
Rounds 45240
Rounds 45250
Rounds 45260
Rounds 45270
Rounds 45280
Rounds 45290
Rounds 45300
Rounds 45310
Rounds 45320
Rounds 45330
Rounds 45340
Rounds 45350
Rounds 45360
Rounds 45370
Rounds 45380
Rounds 45390
Rounds 45400
Rounds 45410
Rounds 45420
Rounds 45430
Rounds 45440
Rounds 45450
Rounds 45460
Rounds 45470
Rounds 45480
Rounds 45490
Rounds 45500
Rounds 45510
Rounds 45520
Rounds 45530
Rounds 45540
Rounds 45550
Rounds 45560
Rounds 45570
Rounds 45580
Rounds 45590
Rounds 45600
Rounds 45610
Rounds 45620
Rounds 45630
Rounds 45640
Rounds 45650
Rounds 45660
Rounds 45670
Rounds 45680
Rounds 45690
Rounds 45700
Rounds 45710
Rounds 45720
Rounds 45730
Rounds 45740
Rounds 45750
Rounds 45760
Rounds 45770
Rounds 45780
Rounds 45790
Rounds 45800
Rounds 45810
Rounds 45820

In [17]:
p1.loadPolicy("policy2_legr_0.2_0.1_0.95_50000_p1")
p2.loadPolicy("policy2_legr_0.2_0.1_0.95_50000_p2")

### Computer vs Computer

In [19]:
p1 = Player("computer1", exp_rate=0)
p1.loadPolicy("policy2_legr_0.2_0.1_0.95_100000_p1")
p2 = Player("computer2", exp_rate=0)
p2.loadPolicy("policy2_legr_0.2_0.1_0.95_100000_p2")

# p1 = HumanPlayer("human1")
# p2 = HumanPlayer("human2")

st = State(p1, p2)
st.play2()

------------------------------
| x |   |   |   |   |   | o | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   | x |   |   | 
------------------------------
| o |   |   |   |   |   |   | 
------------------------------
------------------------------
| x |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   | o |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   | x |   |   | 
------------------------------
| o |   |   |   |   |   |   | 
------------------------------
------------------------------
| x |   |   |   |   |   |   | 
--------

------------------------------
| x | o |   |   |   |   |   | 
------------------------------
|   |   | o | o | o |   |   | 
------------------------------
|   |   | o | o | o |   |   | 
------------------------------
|   |   |   | o |   |   |   | 
------------------------------
|   |   |   | o | o |   |   | 
------------------------------
|   | o | o | o |   |   |   | 
------------------------------
|   | o | o | o |   |   |   | 
------------------------------
------------------------------
| x | x |   |   |   |   |   | 
------------------------------
|   | x | x | o | o |   |   | 
------------------------------
|   |   | x | o | o |   |   | 
------------------------------
|   |   |   | o |   |   |   | 
------------------------------
|   |   |   | o | o |   |   | 
------------------------------
|   | o | o | o |   |   |   | 
------------------------------
|   | o | o | o |   |   |   | 
------------------------------
------------------------------
| x | x |   |   |   |   |   | 
--------

------------------------------
|   | x |   |   |   |   | o | 
------------------------------
|   |   | x | x | x |   |   | 
------------------------------
|   |   | x | x | x |   |   | 
------------------------------
|   |   | x | x | x | x |   | 
------------------------------
|   | o | o | o |   |   |   | 
------------------------------
| o | o | o | o |   |   |   | 
------------------------------
| o | o | o | o |   |   |   | 
------------------------------
------------------------------
|   | x |   |   |   |   | o | 
------------------------------
|   |   | x | x | x |   |   | 
------------------------------
|   |   | x | x | x |   |   | 
------------------------------
|   |   | x | x | x | x |   | 
------------------------------
| o | o | o | o |   |   |   | 
------------------------------
| o | o | o | o |   |   |   | 
------------------------------
|   | o | o | o |   |   |   | 
------------------------------
------------------------------
|   | x |   |   |   |   | o | 
--------

------------------------------
|   | o | o | o | x | x | x | 
------------------------------
|   | o |   | x | x |   |   | 
------------------------------
|   |   | o | x | x |   |   | 
------------------------------
|   |   | x | x |   |   |   | 
------------------------------
| o | x | o | o | o |   |   | 
------------------------------
| x | x | x | o | o |   |   | 
------------------------------
| x | x | x | o | x | x |   | 
------------------------------
------------------------------
|   | o | o | o | x | x | x | 
------------------------------
|   | o | o | o | x |   |   | 
------------------------------
|   |   | o | o | x |   |   | 
------------------------------
|   |   | x | x |   |   |   | 
------------------------------
| o | x | o | o | o |   |   | 
------------------------------
| x | x | x | o | o |   |   | 
------------------------------
| x | x | x | o | x | x |   | 
------------------------------
------------------------------
|   | o | o | o | x | x | x | 
--------

------------------------------
| o | x | x | x | x | x | x | 
------------------------------
| o | o | o | x | x |   | x | 
------------------------------
| o | o |   | o |   |   |   | 
------------------------------
|   |   | o | o |   |   |   | 
------------------------------
| o |   | o | o |   | x |   | 
------------------------------
| o |   | x | x | x | x | x | 
------------------------------
| x | x | x | x | x | x | x | 
------------------------------
------------------------------
| o | x | x | x | x | x | x | 
------------------------------
| o | o | o | o | x |   | x | 
------------------------------
| o | o | o | o |   |   |   | 
------------------------------
|   |   | o | o |   |   |   | 
------------------------------
| o |   | o | o |   | x |   | 
------------------------------
| o |   | x | x | x | x | x | 
------------------------------
| x | x | x | x | x | x | x | 
------------------------------
------------------------------
| o | x | x | x | x | x | x | 
--------